# Roteiro Levenberg-Marquadt

Esse código é um protótipo de função para a implementação do algoritmo de Levenberg-Marquadt, o qual é utilizado para estimar parâmetros não-lineares. 

\begin{equation}
\Gamma (\textbf{m}) = \Psi (\textbf{m}) + \sum\limits^{6}_{j=1}\alpha_{j}\varphi_{j}(\textbf{m})
\end{equation}

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from __future__ import division
from fatiando import mesher, gridder, utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.gravmag import prism
from fatiando.mesher import Prism
from fatiando.vis import mpl, myv
from fatiando.constants import CM, T2NT
from matplotlib import colors, ticker, cm
import mag_polyprism_functions as mfun
import polyprism_tests as tests
% matplotlib inline

C:\Users\Vanderlei\Documents\fatiando\fatiando\vis\mpl.py:70: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")
C:\Users\Vanderlei\Documents\fatiando\fatiando\vis\myv.py:53: UserWarning: This module will be removed in v0.7.
  warnings.warn("This module will be removed in v0.7.")


# Entrada

### Parâmetros para o modelo direto

In [2]:
M = 6 # number of vertices per prism
L = 3 # number of prisms
P = L*(M+2) # number of parameters

inc, dec = -60., 50. # inclination and declination of regional field
incs, decs = 70., -50.      # wrong direction or remanent

#r = 1000. # radial distance for each vertice
r = np.zeros(M)
r[::2] = 3000.
r[1::2] = np.sqrt(2.)*3000.

# Cartesian coordinates of the origin of each prism
x0 = np.zeros(L) 
y0 = np.zeros(L)

dz = 100.0    # thickness of each prism

In [3]:
props={'magnetization': utils.ang2vec(3., inc, dec)} # physical property

z0 = 0.    # depth of the top the shallowest prism

l = []   # list of prisms

### creating the lis of prisms

for i in range(L):
    l.append([r, x0[i], y0[i], z0 + dz*i, z0 + dz*(i + 1), props])

### Pontos de observação

In [4]:
#area over which the data are calculated
#x minimum, x maximum, y minimum and y maximum
area = [-10000, 10000, -10000, 10000] 

#number of data along the y and x directions
shape = (80,80)

#total number of data
N = shape[0]*shape[1]

#coordinates x and y of the data
x = np.linspace(area[0],area[1],shape[0]) # points in x
y = np.linspace(area[2],area[3],shape[0]) # points in y
xp,yp = np.meshgrid(x,y)    # creating mesh points
xp = xp.ravel()
yp = yp.ravel()

#vertical coordinates of the data
zp = -350. - 500.*utils.gaussian2d(xp, yp, 17000, 21000, 21000, 18500, angle=21) # relief

### Parâmetros exigidos durante a inversão

In [5]:
lamb = 0.001 # Marquadt's parameter
dlamb = 5.  # step for Marquadt's parameter

# limits for parameters in meters
rmin = 100.
rmax = 6000.
x0min = -5000.
x0max = 5000.
y0min = -5000.
y0max = 5000.

In [6]:
mmax = np.zeros(M+2)
mmin = np.zeros(M+2)

mmax[:M] = rmax
mmax[M] = x0max
mmax[M+1] = y0max
mmin[:M] = rmin
mmin[M] = x0min
mmin[M+1] = y0min

print mmax
print mmin

mmax = np.resize(mmax, P)
mmin = np.resize(mmin, P)

print mmax
print mmin

[ 6000.  6000.  6000.  6000.  6000.  6000.  5000.  5000.]
[  100.   100.   100.   100.   100.   100. -5000. -5000.]
[ 6000.  6000.  6000.  6000.  6000.  6000.  5000.  5000.  6000.  6000.
  6000.  6000.  6000.  6000.  5000.  5000.  6000.  6000.  6000.  6000.
  6000.  6000.  5000.  5000.]
[  100.   100.   100.   100.   100.   100. -5000. -5000.   100.   100.
   100.   100.   100.   100. -5000. -5000.   100.   100.   100.   100.
   100.   100. -5000. -5000.]


In [7]:
delta = 10. # increment of derivatives

m_out = np.zeros(M + 2)  # outcropping body parameters

preg = 0.1000 # regularization parameter

# Gerando os dados preditos

In [8]:
m = mfun.param_vec(l, M, L) # true parameters vector

In [9]:
m

array([ 3000.        ,  4242.64068712,  3000.        ,  4242.64068712,
        3000.        ,  4242.64068712,     0.        ,     0.        ,
        3000.        ,  4242.64068712,  3000.        ,  4242.64068712,
        3000.        ,  4242.64068712,     0.        ,     0.        ,
        3000.        ,  4242.64068712,  3000.        ,  4242.64068712,
        3000.        ,  4242.64068712,     0.        ,     0.        ])

In [10]:
model = mfun.pol2cart(l, M, L) # list of classes of prisms

In [11]:
d = polyprism.tf(xp, yp, zp, model, inc, dec) # predict data

In [1]:
plt.figure(figsize=(15,10))
plt.subplot(1,3,1)
plt.title("Predicted data (nT)", fontsize = 18)
plt.axis('scaled')
nlevels = mpl.contourf(yp, xp, d,shape, 15, cmap='RdBu_r')
plt.xlabel('East y (km)')
plt.ylabel('North x (km)')
plt.colorbar(pad=0.01, aspect=20, shrink=0.4).set_label('nT')
for p in model:
    mpl.polygon(p, '.-k', xy2ne=True)
mpl.m2km()
plt.show()

NameError: name 'plt' is not defined

# Algoritmo Levenberg-Marquadt

1- Iteração $k=0$: chute inicial $\mathbf{m}_0$ em que todos os elementos satisfaçam os vínculos de desigualdade e um valor real positivo grande para $\lambda$

2) Calcula os parâmetros transformados $\breve{\mathbf{m}}_k$

Os parâmetros transformados são calculados a partir da equação:

\begin{equation}
\breve{m}_i = f(m_i) = - \ln\left(\frac{m_{i\, max} - m_i}{m_i - m_{i\, min}}\right)
\end{equation}

A transformação inversa dos parâmetros é dada pela equação:

\begin{equation}
m_i = f^{-1}(\breve{m}_i) = m_{i\, min} + \left(\frac{m_{i\, max} - m_{i\, min}}{1 + e^{-\breve{m}_i}}\right)
\end{equation}

3) laço externo: calcula $\nabla^2\Gamma (\hat{\mathbf{m}}_k)$, $\nabla\Gamma (\hat{\mathbf{m}}_k)$ e $T(\hat{\mathbf{m}}_k)$:

In [13]:
#props={'magnetization': utils.ang2vec(3, inc, dec)} # physical property

#z0 = 0.    # depth of the top the shallowest prism

rini = np.full((M),(5000.))

lini = []   # list of parameters of each prisms

### creating the list of prisms
for i in range(L):
    lini.append([rini, x0[i], y0[i], z0 + dz*i, z0 + dz*(i + 1), props])

In [14]:
alpha1 = 0.000001
alpha2 = alpha1
alpha3 = 0.
alpha4 = 0.
alpha5 = alpha1
alpha6 = alpha1

In [16]:
itmax = 50
itmax_marq = 15
epsilon = 0.001
lamb = 0.001
dlamb = 5.
l0 = deepcopy(lini)
m0 = mfun.param_vec(l0, M, L) # inicial parameters vector
model0 = mfun.pol2cart(l0, M, L) # list of classes of prisms
d0 = polyprism.tf(xp, yp, zp, model0, inc, dec) # predict data
dif = d - d0
phi0 = np.sum(dif*dif)/xp.size
phi_list = [phi0]

i,j = np.diag_indices(P)

for it in range(itmax):
    #mt = mfun.trans_parameter(m0, M, L, rmin, rmax, x0min, x0max, y0min, y0max) # transformated parameters vector
    mt = mfun.trans_parameter2(m0, mmax, mmin)
    
    grad = mfun.gradient_data(xp, yp, zp, l0, M, L, d, delta, delta, delta, inc, dec)
    grad = mfun.gradient_phi_1(M, L, grad, alpha1)
    grad = mfun.gradient_phi_2(M, L, grad, alpha2)
    grad = mfun.gradient_phi_3(M, L, grad, m_out, alpha3)
    grad = mfun.gradient_phi_4(M, L, grad, m_out[-2:], alpha4)
    grad = mfun.gradient_phi_5(M, L, grad, alpha5)
    grad = mfun.gradient_phi_6(M, L, grad, alpha6)
    
    H = mfun.Hessian_data(xp, yp, zp, l0, M, L, delta, delta, delta, inc, dec)
    H = mfun.Hessian_phi_1(M, L, H, alpha1)
    H = mfun.Hessian_phi_2(M, L, H, alpha2)
    H = mfun.Hessian_phi_3(M, L, H, alpha3)
    H = mfun.Hessian_phi_4(M, L, H, alpha4)
    H = mfun.Hessian_phi_5(M, L, H, alpha5)
    H = mfun.Hessian_phi_6(M, L, H, alpha6)
    
    # Diagonal da matriz de positividade
    T = ((mmax - m0)*(m0 - mmin))/(mmax - mmin)
    
    for it_marq in range(itmax_marq): 
        
        S = H.copy()
        S[i,:] *= T
        S[i,j] += lamb
    
        delta_mt = np.linalg.solve(S, -grad)
        #m_est = mfun.trans_inv_parameter(mt + delta_mt, M, L, rmin, rmax, x0min, x0max, y0min, y0max)
        m_est = mfun.trans_inv_parameter2(mt + delta_mt, mmax, mmin)
        l_est = mfun.param2model(m_est, M, L, z0, dz, props)
        d_est = polyprism.tf(xp, yp, zp, l_est, inc, dec)
        res = d - d0
        phi = np.sum(res*res)/xp.size
        phi += (mfun.phi_1(M, L, m_est, alpha1) + mfun.phi_2(M, L, m_est, alpha2) + \
                mfun.phi_3(M, L, m_est, m_out, alpha3) + mfun.phi_4(M, L, m_est, m_out[-2:], alpha4) + \
                mfun.phi_5(M, L, m_est, alpha5) + mfun.phi_6(M, L, m_est, alpha6))
        
        dphi = phi - phi0

        print it, phi0, it_marq, phi, dphi, lamb
        
        if (dphi > 0.):
            lamb *= dlamb
        else:
            lamb /= dlamb
            
    if ((phi0 - phi)/phi0 > epsilon):
        d0 = d_est
        m0 = m_est
        l0 = l_est
        phi0 = phi
        phi_list.append(phi0)
    else:
        break

0 5086.23152721 0 1673666932.36 1673661846.13 0.001
0 5086.23152721 1 1674697122.28 1674692036.05 0.005
0 5086.23152721 2 1675773004.04 1675767917.81 0.025
0 5086.23152721 3 1676344786.16 1676339699.92 0.125
0 5086.23152721 4 1676716135.76 1676711049.53 0.625
0 5086.23152721 5 1676938419.65 1676933333.42 3.125
0 5086.23152721 6 1677009537.19 1677004450.96 15.625
0 5086.23152721 7 1677025733.79 1677020647.56 78.125
0 5086.23152721 8 1677029078.24 1677023992.0 390.625
0 5086.23152721 9 1677029751.94 1677024665.71 1953.125
0 5086.23152721 10 1677029886.88 1677024800.65 9765.625
0 5086.23152721 11 1677029913.87 1677024827.64 48828.125
0 5086.23152721 12 1677029919.27 1677024833.04 244140.625
0 5086.23152721 13 1677029920.35 1677024834.12 1220703.125
0 5086.23152721 14 1677029920.57 1677024834.34 6103515.625


In [ ]:
phi_list

In [ ]:
mt

In [ ]:
np.exp(-mt)

In [ ]:
lamb

In [ ]:
np.log(1./0.)

In [ ]:
mpl.figure(figsize=(15,10))
mpl.subplot(1,3,1)
mpl.title("Predicted data (nT)", fontsize = 18)
mpl.axis('scaled')
nlevels = mpl.contourf(yp, xp, d0,shape, 15, cmap='RdBu_r')
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar(pad=0.01, aspect=20, shrink=0.4).set_label('nT')
mpl.m2km()
mpl.show()

In [ ]:
ddif = d - d0
mpl.figure(figsize=(15,10))
mpl.subplot(1,3,1)
mpl.title("Predicted data (nT)", fontsize = 18)
mpl.axis('scaled')
nlevels = mpl.contourf(yp, xp, ddif,shape, 15, cmap='RdBu_r')
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar(pad=0.01, aspect=20, shrink=0.4).set_label('nT')
mpl.m2km()
mpl.show()

In [ ]:
#m_volta = mfun.trans_inv_parameter(mt, M, L, rmin, rmax, x0min, x0max, y0min, y0max) # inverse transformation

#np.allclose(m_volta, m0) # validation test

# creating a model from the transformated parameters vector

#ltrans = [] # list of parameters of each prisms

#ii = i*(M+2)

### creating the list of prisms
#for i in range(L):
#    ltrans.append([mt[ii:ii+M], mt[ii+M], mt[ii+(M+1)], z0 + dz*i, z0 + dz*(i + 1), props])

#modeltrans = mfun.pol2cart(ltrans, M, L)

In [ ]:
m0[4:6] = 500.

In [ ]:
mp = mfun.trans_parameter(m0, M, L, rmin, rmax, x0min, x0max, y0min, y0max)

In [ ]:
mv = mfun.trans_inv_parameter(mp, M, L, rmin, rmax, x0min, x0max, y0min, y0max)

In [ ]:
mp

In [ ]:
mv

In [ ]:
mtt = np.zeros_like(m)

for i in range(0, P, M+2):
        mtt[i:M+i] = - np.log((rmax - m0[i:M+i])/(m0[i:M+i] - rmin))
        mtt[i+M] = - np.log((x0max - m0[i+M])/(m0[i+M] - x0min))
        mtt[i+M+1] = - np.log((y0max - m0[i+M+1])/(m0[i+M+1] - y0min))
        print (x0max - m0[i+M]), (m0[i+M] - x0min)
        print (y0max - m0[i+M+1]), (m0[i+M+1] - y0min)

In [ ]:
- np.log(31316.0714668/11316.0714668)

In [ ]:
mtt

a) laço interno: calcula $\Delta \breve{\mathbf{m}}_k$ solucionando o sistema linear abaixo

\begin{equation}
\left[\nabla^2\Gamma (\hat{\mathbf{m}}_k) + \lambda \mathbf{I} \right]\mathbf{T}(\hat{\mathbf{m}}_k) \Delta\breve{\mathbf{m}}_k \approx - \nabla \Gamma (\hat{\mathbf{m}}_k)
\end{equation}

onde $\mathbf{T}(\hat{\mathbf{m}}_k)$ é uma matriz diagonal
\begin{equation}
[\mathbf{T}(\hat{\mathbf{m}}_k)]_{ii} = \frac{(m_{i\, max} - m_i)(m_i - m_{i\, min})}{(m_{i\, max} - m_{i\, min})}
\end{equation}

b) calcula $\breve{\mathbf{m}}_{k+1} = \breve{\mathbf{m}}_k + \Delta\breve{\mathbf{m}}_k$

c) computa $\hat{\mathbf{m}}_{k+1}$

d) Enquanto $\Gamma (\hat{\mathbf{m}}_{k+1})<\Gamma (\hat{\mathbf{m}}_k)$ 

   i $\lambda = \lambda\cdot d\lambda$

   ii retorne para 3)a)

e) $\lambda = \lambda/d\lambda$

4) Enquanto $\Gamma (\hat{\mathbf{m}}_{k+1})<\Gamma (\hat{\mathbf{m}}_k)$ e $\frac{|\Gamma (\hat{\mathbf{m}}_{k+1})- \Gamma (\hat{\mathbf{m}}_k)|}{|\Gamma (\hat{\mathbf{m}}_k)|}>\varepsilon$

a) $k=k+1$

b) Retorne para etapa 2)

In [ ]:
P

In [ ]:
(10000. + 100.*np.exp(0.020202073175))/(np.exp(0.020202073175)+1)

In [ ]:
100. + 9900./(np.exp(0.020202073175)+1)